Author: Mario Gennaro

"""
This notebook reads the Oxigen-enhanced VanDenberg isochrones
computed by Tom Brown for his 2014 paper on UFDs (F606W, F814W)
and stores the whole grid as a pandas dataframe using shelve.
It also computes and stores scipy interpolators for the same database
"""

In [ ]:
import pandas as pd
import shelve
import glob
import re
import numpy as np
import pickle
from synCMD.auxfunc.photband import photband
from scipy.interpolate import interp1d
from scipy.interpolate import LinearNDInterpolator
from TSMC_utils.intNN import intNN

In [ ]:
# List all the iscrone files in the directory,
# read them and put them in a pandas dataframe
# in case of trouble check this:
# http://stackoverflow.com/questions/13226029/benefits-of-pandas-multiindex


flist = glob.glob('/user/gennaro/UFDs_OPT/isocrones/f*')
dfACSiso = pd.DataFrame()

for file in flist:
    metchar = (re.search('fm(.+?)a',file)).group(1)
    agechar = (re.search(metchar+'a(.+?)\.iso',file)).group(1)
    met = -1.*np.asarray(metchar,dtype=float)/100.
    age = np.asarray(agechar,dtype=float)/100.

    if age == 11:
        print('Metallicity: ',met)
    
    num_lines = sum(1 for line in open(file))
    df1 = pd.read_table(file,header=None,sep='\s+',
          names=['mass','logT','logL','logg','F606W','F814W'],index_col=['mass'])
    arrays = [np.repeat(met,num_lines),np.repeat(age,num_lines),df1.index]
    ind = pd.MultiIndex.from_tuples(list(zip(*arrays)),names=['[Fe/H]', 'age','mass'])
    df1.index = ind
    
    dfACSiso = dfACSiso.append(df1)

In [ ]:
# Bands to use

photbands = []  #List of photband objects

pb1 = photband()
pb1.name = 'F606W'
pb1.lowcut = 90.

photbands.append(pb1)

pb1 = photband()
pb1.name = 'F814W'
pb1.lowcut  = 90.

photbands.append(pb1)

In [ ]:
# Get the available values of age and metallicity

age_vals = np.unique(np.asarray([dfACSiso.index.get_level_values('age')]).T)
feh_vals = np.unique(np.asarray([dfACSiso.index.get_level_values('[Fe/H]')]).T)

In [ ]:
# Create alist of interpolators (age X mets x bands)
# these can be used for NN interpolation in age and met (i.e. find the closest isochrone in age and met)
# and linear interpolation in mass

iso_intp = [[0 for x in range(len(feh_vals))] for y in range(len(age_vals))]
iso_mrng = [[0 for x in range(len(feh_vals))] for y in range(len(age_vals))]

for aa, age in enumerate(age_vals):
    print("Doing age",age)
    for zz, met in enumerate(feh_vals):
        isomass = np.asarray(dfACSiso.ix[met].ix[age].index.get_level_values('mass'))
        iso_mrng[aa][zz] = ([np.amin(isomass),np.amax(isomass)])
        iso1 = []
        for pb in photbands:
            isomag = np.asarray(dfACSiso.ix[met].ix[age][pb.name])
            iso1.append(interp1d(isomass, isomag, kind='linear', assume_sorted=True))
        iso_intp[aa][zz] = iso1

In [ ]:
# Create an ND linear interpolator (mass,age,metallicity)

points = np.asarray([dfACSiso.index.get_level_values(2),
                     dfACSiso.index.get_level_values(1),
                     dfACSiso.index.get_level_values(0)]).T

iso_LNDInt = []
for pb in photbands:
    print("Doing ND linear interpolator for band",pb.name)
    iso_LNDInt.append(LinearNDInterpolator(points,dfACSiso[pb.name], fill_value=np.nan, rescale=True))


In [ ]:
# Create a NN (age,metallicity) + linear interpolator (mass)
print("Doing NN linear interpolator")

iso_NNInt =[]
for photband in photbands:
    iso_NNInt.append(intNN(dfACSiso,photband))

In [ ]:
#Now save the dataframe and the interpolators in a shelf

isopd      = {'dfACSiso':dfACSiso,
              'age_vals':age_vals,
              'feh_vals':feh_vals}

pickle.dump(isopd, open( '/user/gennaro/UFDs_OPT/shelves/isoACS.pickle', "wb" ) )   

isoNN_v0 = {'iso_intp':iso_intp,
            'iso_mrng':iso_mrng}

pickle.dump(isoNN_v0 , open( '/user/gennaro/UFDs_OPT/shelves/isoNN_v0.pickle', "wb" ) )
 

In [ ]:
pickle.dump(iso_LNDInt , open( '/user/gennaro/UFDs_OPT/shelves/iso_LNDint.pickle', "wb" ) )        

In [ ]:
pickle.dump(iso_NNInt , open( '/user/gennaro/UFDs_OPT/shelves/iso_NNint.pickle', "wb" ) )